In [1]:
import pandas as pd
import configparser
config = configparser.ConfigParser()
import sys
config.read("../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']
sys.path.append(local_lib)
from textparser import TextParser

OHCO = ['speech_id', 'para_num', 'sent_num', 'token_num']

In [2]:
LIB = pd.read_csv('LIB.csv', sep='|').set_index('speech_id')
VOCAB = pd.read_csv('VOCAB.csv', sep='|').set_index('term_str')
BOW = pd.read_csv('BOW.csv').set_index(['speech_id', 'para_num'])

In [3]:
salex = pd.read_csv('salex_nrc.csv').set_index('term_str')
salex.columns = [i.split('_')[1] for i in salex.columns]
salex

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,sentiment
term_str,,,,,,,,,,,
abandon,0,0,0,1,0,1,0,1,0,0,-1
abandoned,1,0,0,1,0,1,0,1,0,0,-1
abandonment,1,0,0,1,0,1,0,1,1,0,-1
abduction,0,0,0,1,0,1,0,1,1,0,-1
aberration,0,0,1,0,0,1,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...
young,0,1,0,0,1,0,1,0,1,0,1
youth,1,1,0,1,1,0,1,0,1,0,1
zeal,0,1,0,0,1,0,1,0,1,1,1


In [4]:
VOCAB_SENT = pd.concat([VOCAB.reset_index().set_index('term_str'), salex], join='inner', axis=1)
VOCAB_SENT

,n,n_chars,p,i,max_pos,max_pos_group,stop,porter_stem,dfidf,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,sentiment
term_str,,,,,,,,,,,,,,,,,,,,
abandon,29,7,0.000053,14.194818,VB,VB,0,abandon,241.869006,0,0,0,1,0,1,0,1,0,0,-1
abandoned,14,9,0.000026,15.245444,VBN,VB,0,abandon,131.473113,1,0,0,1,0,1,0,1,0,0,-1
abandonment,3,11,0.000006,17.467836,NN,NN,0,abandon,34.839987,1,0,0,1,0,1,0,1,1,0,-1
abduction,1,9,0.000002,19.052799,NN,NN,0,abduct,13.198292,0,0,0,1,0,1,0,1,1,0,-1
abhor,1,5,0.000002,19.052799,VBP,VB,0,abhor,13.198292,1,0,1,1,0,1,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
young,202,5,0.000371,11.394587,JJ,JJ,0,young,1014.331423,0,1,0,0,1,0,1,0,1,0,1
youth,35,5,0.000064,13.923516,NN,NN,0,youth,262.345330,1,1,0,1,1,0,1,0,1,0,1
zeal,7,4,0.000013,16.245444,NN,NN,0,zeal,72.736556,0,1,0,0,1,0,1,0,1,1,1


In [5]:
emo_cols = VOCAB_SENT.columns[-11:]
col_list = ['max_pos']
for i in emo_cols:
    col_list.append(i)
BOW_SENT = BOW.join(VOCAB_SENT[col_list], on='term_str').dropna()
BOW_SENT

term_str  n     tfidf max_pos  anger  anticipation  \
speech_id para_num                                                         
0         0             fellow  1  0.631399      JJ    0.0           0.0   
          1         auspicious  1  0.130487      JJ    0.0           1.0   
          1            blessed  1  0.105516     VBN    0.0           0.0   
          1            concord  1  0.122206      NN    0.0           0.0   
          1             credit  1  0.068781      NN    0.0           0.0   
...                        ... ..       ...     ...    ...           ...   
113       264            white  1  0.084893     NNP    0.0           1.0   
          265        president  1  0.081955     NNP    0.0           0.0   
          266           united  1  0.133458     NNP    0.0           0.0   
          267            bless  1  0.470367      NN    0.0           1.0   
          267              god  2  0.766525     NNP    0.0           1.0   

                    disgust  fear  joy  negative  positive  sadness  surprise  \
speech_id para_num                                                              
0         0             0.0   0.0  0.0       0.0       1.0      0.0       0.0   
          1             0.0   0.0  1.0       0.0       1.0      0.0       0.0   
          1             0.0   0.0  1.0       0.0       1.0      0.0       0.0   
          1             0.0   0.0  0.0       0.0       1.0      0.0       0.0   
          1             0.0   0.0  0.0       0.0       1.0      0.0       0.0   
...                     ...   ...  ...       ...       ...      ...       ...   
113       264           0.0   0.0  1.0       0.0       1.0      0.0       0.0   
          265           0.0   0.0  0.0       0.0       1.0      0.0       0.0   
          266           0.0   0.0  0.0       0.0       1.0      0.0       0.0   
          267           0.0   0.0  1.0       0.0       1.0      0.0       0.0   
          267           0.0   1.0  1.0       0.0       1.0      0.0       0.0   

                    trust  sentiment  
speech_id para_num                    
0         0           1.0        1.0  
          1           0.0        1.0  
          1           0.0        1.0  
          1           1.0        1.0  
          1           1.0        1.0  
...                   ...        ...  
113       264         1.0        1.0  
          265         1.0        1.0  
          266         1.0        1.0  
          267         1.0        1.0  
          267         1.0        1.0  

[38311 rows x 15 columns]

In [6]:
DOC_SENT = BOW_SENT.reset_index().groupby('speech_id')[emo_cols].mean()
DOC_SENT.index = LIB.reset_index().speech_id
DOC_SENT

,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust,sentiment
speech_id,,,,,,,,,,,
1,0.079545,0.443182,0.022727,0.170455,0.318182,0.136364,0.863636,0.045455,0.159091,0.647727,0.727273
2,0.130000,0.300000,0.050000,0.220000,0.390000,0.260000,0.710000,0.160000,0.130000,0.610000,0.450000
3,0.131737,0.317365,0.065868,0.209581,0.299401,0.263473,0.748503,0.125749,0.107784,0.556886,0.485030
4,0.155405,0.310811,0.108108,0.229730,0.277027,0.310811,0.628378,0.155405,0.094595,0.594595,0.317568
5,0.203704,0.265432,0.123457,0.240741,0.253086,0.339506,0.623457,0.179012,0.080247,0.524691,0.283951
...,...,...,...,...,...,...,...,...,...,...,...
110,0.231733,0.329854,0.177453,0.323591,0.359081,0.373695,0.603340,0.208768,0.164927,0.465553,0.229645
111,0.214123,0.323462,0.134396,0.287016,0.337130,0.375854,0.608200,0.220957,0.161731,0.487472,0.232346
112,0.230290,0.296680,0.141079,0.304979,0.280083,0.400415,0.585062,0.236515,0.130705,0.497925,0.184647


In [7]:
VOCAB_SENT.to_csv('VOCAB_SENT.csv')
BOW_SENT.to_csv('BOW_SENT.csv')
DOC_SENT.to_csv('DOC_SENT.csv')